# Unicode code points

Unicode code points are assigning an integer to every character and script across different writing systems. 

In [4]:
ord("N") #using the ord function in python we can display the unicode encoding of a character

78

We cannot use the unicode code points standard as our vocabulary for a LLM. First of all because we would end up with a gigantic vocabulary size but also because unicode is not a stable representation of the characters as it keeps on being updated. 

# UTF-8

One encoding standard for representing every character in the unicode standard is UTF-8 which uses 1 to 4 bytes per character. The first 128 character of UTF-8 are the same than ASCII making UTF-8 backward compatible with ASCII. 


In [5]:
list("Hello World!".encode("utf-8"))

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33]

We cannot keep this as our vocabulary because we only a certain context windown size for our transformer and keeping utf_! just like so would make us end up with very long sequences adn therefore end up with an attention becoming extremely expensive. That's why we will use the byte pair encoding algorithm to leverage compression.

# The Byte Pair Encoding Algorithm

The Byte Pair Encoding Algorithm rely on the following steps. Given a sequence of tokens and a vocabulary set, we are looking for the pair of juxtaposed tokens in the sequence occuring the most often. Once identified, we are creating a new token which is the concatenation of these two juxtaposed token (therefore augmenting our vocabulary size) and replacing them with our new token in the sequence. Thus the sequence is reducing in length and the vocabulary size is increasing. 

In [1]:
text = "We are at the very beginning of time for the human race. It is not unreasonable that we grapple with problems. But there are tens of thousands of years in the future. Our responsibility is to do what we can, learn what we can, improve the solutions, and pass them on. 🔭"
tokens = list(text.encode("utf-8"))

print(text)
print("Length of text :", len(text))
print("-------------------")

print(tokens)
print("Length of tokens :", len(tokens))


We are at the very beginning of time for the human race. It is not unreasonable that we grapple with problems. But there are tens of thousands of years in the future. Our responsibility is to do what we can, learn what we can, improve the solutions, and pass them on. 🔭
Length of text : 269
-------------------
[87, 101, 32, 97, 114, 101, 32, 97, 116, 32, 116, 104, 101, 32, 118, 101, 114, 121, 32, 98, 101, 103, 105, 110, 110, 105, 110, 103, 32, 111, 102, 32, 116, 105, 109, 101, 32, 102, 111, 114, 32, 116, 104, 101, 32, 104, 117, 109, 97, 110, 32, 114, 97, 99, 101, 46, 32, 73, 116, 32, 105, 115, 32, 110, 111, 116, 32, 117, 110, 114, 101, 97, 115, 111, 110, 97, 98, 108, 101, 32, 116, 104, 97, 116, 32, 119, 101, 32, 103, 114, 97, 112, 112, 108, 101, 32, 119, 105, 116, 104, 32, 112, 114, 111, 98, 108, 101, 109, 115, 46, 32, 66, 117, 116, 32, 116, 104, 101, 114, 101, 32, 97, 114, 101, 32, 116, 101, 110, 115, 32, 111, 102, 32, 116, 104, 111, 117, 115, 97, 110, 100, 115, 32, 111, 102, 32, 121, 

Here we can see that the length of the raw text and the length of the tokens are different because some characters are encoded using more than 1 byte. ASCII character are only taking 1 byte but others like emojis take more. 


In [18]:
# Function which for a given sequence of tokens is outputing the stats of each pair of tokens, i.e the number of occurrence for each pair
def find_pair_stats(tokens):
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        if pair in counts:
            counts[pair]+=1
        else :
            counts[pair]=1
    reversed_counts = {}
    for pair, count in counts.items():
        if count in reversed_counts:
            reversed_counts[count].append(pair)
        else : 
            reversed_counts[count] = [pair]
    return reversed_counts

In [19]:
find_pair_stats(tokens)

{1: [(87, 101),
  (32, 118),
  (114, 121),
  (32, 98),
  (98, 101),
  (101, 103),
  (103, 105),
  (110, 110),
  (110, 105),
  (110, 103),
  (103, 32),
  (109, 101),
  (102, 111),
  (111, 114),
  (32, 104),
  (104, 117),
  (117, 109),
  (109, 97),
  (97, 99),
  (99, 101),
  (32, 73),
  (73, 116),
  (32, 110),
  (110, 111),
  (111, 116),
  (32, 117),
  (117, 110),
  (110, 114),
  (110, 97),
  (97, 98),
  (32, 103),
  (103, 114),
  (97, 112),
  (112, 112),
  (112, 108),
  (119, 105),
  (104, 32),
  (111, 98),
  (109, 115),
  (115, 46),
  (32, 66),
  (66, 117),
  (116, 101),
  (101, 110),
  (104, 111),
  (111, 117),
  (117, 115),
  (115, 97),
  (100, 115),
  (32, 121),
  (121, 101),
  (114, 115),
  (102, 117),
  (116, 117),
  (32, 79),
  (79, 117),
  (101, 115),
  (115, 112),
  (112, 111),
  (115, 105),
  (105, 98),
  (98, 105),
  (105, 108),
  (108, 105),
  (116, 121),
  (116, 111),
  (32, 100),
  (100, 111),
  (32, 108),
  (114, 110),
  (109, 112),
  (111, 118),
  (32, 115),
  (111, 108)

In [32]:
def find_most_common_pair(dictionary):
    max = 0
    for count, pair in dictionary.items():
        if count > max :
            max = count
    return dictionary[max], ''.join(chr(i) for i in dictionary[max][0])

In [33]:
dictionary = find_pair_stats(tokens)
find_most_common_pair(dictionary)

([(101, 32)], 'e ')